In [ ]:
# https://zhuanlan.zhihu.com/p/24874356260

#################################################### <start> 配置 HuggingFace 和 WandB <start> ####################################################
# from huggingface_hub import login
# hf_token = "XXX"  #到官网免费申请一个
# login(hf_token)

# import wandb
# wb_token = "XXX"  #到官网免费申请一个
# wandb.login(key=wb_token)
# run = wandb.init(
#     project='fine-tune-DeepSeek-R1-Distill-Qwen-14B on xhs Dataset',
#     job_type="training",
#     anonymous="allow"
# )
#################################################### </end> 配置 HuggingFace 和 WandB </end> ####################################################
#************************************************************************************************************************************************

#################################################### <start> 加载模型和分词器 <start> ####################################################
#************************************************************************************************************************************************
# from unsloth import FastLanguageModel

# max_seq_length = 2048
# dtype = None
# load_in_4bit = True
# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name = "unsloth/DeepSeek-R1-Distill-Qwen-14B",
#     max_seq_length = max_seq_length,
#     dtype = dtype,
#     load_in_4bit = load_in_4bit,  # 4bit量化加载
#     token = hf_token,
# )
#################################################### </end> 加载模型和分词器 </end> ####################################################
#************************************************************************************************************************************************

#################################################### <start> 训练模版 <start> ####################################################
#************************************************************************************************************************************************

train_prompt_style = """
以下是一项任务说明，并附带了更详细的背景信息。
请撰写一个满足完成请求的回复。
在回答之前，请仔细考虑问题，并创建一个逐步的思考链，以确保逻辑和准确的回答。

### Instruction:
你是一个资深的小红书文案专家
请你根据以下问题完成写作
### Question:
{}
### Response:
<think>
{}
</think>n:
"""
# EOS_TOKEN = tokenizer.eos_token
#################################################### </end> 训练模版 </end> ####################################################
#************************************************************************************************************************************************

#################################################### <start> 定义函数，让数据集填入模版 <start> ####################################################
#************************************************************************************************************************************************

import re  # 导入正则表达式模块
# 使用正则提取思考链和最终响应
def formatting_prompts_func(examples):
    # 提取数据集里的 instructions 和 outputs （本次数据集的 think 部分融合在 outputs 里）
    instructions = examples["instruction"]
    outputs = examples["output"]
    texts = []
    for instruction_text, output_text in zip(instructions, outputs):
        # 使用正则表达式提取 outputs 里 <think> 和 </think> 之间的内容作为 cots
        match = re.search(r"<think>(.*?)</think>", output_text, re.DOTALL)
        cot = match.group(1).strip() if match else ""

        # 提取 </think> 之后的内容作为 outputs
        output = output_text.replace(match.group(0), "").strip() if match else output_text.strip()

        #text = train_prompt_style.format(instruction_text, cot, output) + EOS_TOKEN
        text = train_prompt_style.format(instruction_text, cot, output) 
        texts.append(text)
    return {"text": texts}
#################################################### </end> 定义函数，让数据集填入模版 </end> ####################################################
#************************************************************************************************************************************************

#################################################### <start> 读取数据集，应用上面定义的函数 <start> ####################################################
#************************************************************************************************************************************************

from datasets import load_dataset

data_files = r'E:\pythonCode\FineTune\XiaoHongShu\Dataset\xhs_data.json'
dataset = load_dataset("json", data_files=data_files, split="train")

# 使用 map 函数进行数据处理
dataset = dataset.map(formatting_prompts_func, batched=True)
#################################################### </end> 读取数据集，应用上面定义的函数 </end> ####################################################
#************************************************************************************************************************************************



In [7]:
from datasets import load_dataset
import re

data_files = r'E:\pythonCode\FineTune\XiaoHongShu\Dataset\xhs_data.json'
dataset = load_dataset("json", data_files=data_files, split="train")
instructions = dataset["instruction"]
outputs = dataset["output"]
match = re.search(r"<think>(.*?)</think>", outputs[0], re.DOTALL)
match
# print("instructions:", instructions[0])
# print("outputs:", outputs[0])
# print("zip:",zip(instructions, outputs))
#for instruction_text, output_text in zip(instructions, outputs):
    #print("instruction_text:", instruction_text)
    #print("output_text:", output_text)
    # match = re.search(r"<think>(.*?)</think>", output_text, re.DOTALL)
    # print("match:", match)
    # break

<re.Match object; span=(0, 798), match='<think>\n好的，用户让我写一篇小红书风格的帖子，标题是“🍑科颜氏男士护肤系列使用心得”。首>